**<h2>Import libaries</h2>**

In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Image processing
from PIL import Image
import cv2

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# Display settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
np.set_printoptions(suppress=True)

print("All libraries imported successfully")


**<h2>Dataset setup</h2>**

In [ ]:
dataset_path = "/kaggle/input/rare-and-endangered-medicinal-plants-in-bangladesh/Raw-Images"

# Verify the dataset exists
if os.path.exists(dataset_path):
    print(f"Dataset found at: {dataset_path}")
else:
    print(f"Dataset not found at: {dataset_path}")
    # Try alternative path (common variations)
    dataset_path = "/kaggle/input/remp-dataset/Raw-Images"
    if os.path.exists(dataset_path):
        print(f"Dataset found at alternative path: {dataset_path}")
    else:
        print("Please check your dataset path")


**<h2>Image Reading</h2>**

In [ ]:
print("\nTesting image reading:\n")
sample_success = 0
sample_failed = 0

for class_name in classes[:3]:  # Test first 3 classes
    class_path = os.path.join(dataset_path, class_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if images:
        sample_path = os.path.join(class_path, images[0])
        try:
            with Image.open(sample_path) as img:
                width, height = img.size
                print(f"  {class_name}: Sample image {width}x{height} - Read successfully")
                sample_success += 1
        except Exception as e:
            print(f"  {class_name}: Failed to read - {e}")
            sample_failed += 1

print(f"\nImage reading test: {sample_success} successful, {sample_failed} failed")

**<h2>Class Balance Analysis</h2>**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Mapping based on file names
plant_mapping = {
    'type_1': 'Kalochitra',
    'type_2': 'Anantamu', 
    'type_3': 'Apang',
    'type_4': 'Syspan',
    'type_5': 'Appishikha',
    'type_6': 'Kalamegh',
    'type_7': 'Kalothutura',
    'type_8': 'Gshnru',
    'type_9': 'Chapalish',
    'type_10': 'Beta',
    'type_11': 'Punarnava',
    'type_12': 'Bamonhati',
    'type_13': 'Basok',
    'type_14': 'Ramtulsi',
    'type_15': 'Shotomuli',
    'type_16': 'Sarpagandha'
}

# Create DataFrame for analysis
plant_data = []
for type_name, plant_name in plant_mapping.items():
    count = class_counts[type_name]  # From previous code
    plant_data.append({'Type': type_name, 'Plant_Name': plant_name, 'Count': count})

df_plants = pd.DataFrame(plant_data)

# Sort by count for better visualization
df_plants = df_plants.sort_values('Count', ascending=False)

print("\033[1mPLANT CLASS DISTRIBUTION ANALYSIS:\033[0m")
print("")

print(f"Total Plants: {len(df_plants)}")
print(f"Total Images: {df_plants['Count'].sum()}")
print(f"Average images per plant: {df_plants['Count'].mean():.1f}")
print(f"Most represented: {df_plants.iloc[0]['Plant_Name']} ({df_plants.iloc[0]['Count']} images)")
print(f"Least represented: {df_plants.iloc[-1]['Plant_Name']} ({df_plants.iloc[-1]['Count']} images)")

print("\033[1m\nIMAGE DIMENSION STATISTICS:\033[0m")
print("")
widths = []
heights = []
aspect_ratios = []
file_sizes = []

# Collect actual image statistics
for class_name in classes:
    class_path = os.path.join(dataset_path, class_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if images:
        sample_path = os.path.join(class_path, images[0])
        try:
            with Image.open(sample_path) as img:
                width, height = img.size
                widths.append(width)
                heights.append(height)
                aspect_ratios.append(width / height)
                file_sizes.append(os.path.getsize(sample_path) / 1024)  # KB
        except Exception as e:
            print(f"Error reading image: {e}")

if widths and heights:
    print(f"Average dimensions: {np.mean(widths):.0f} x {np.mean(heights):.0f} pixels")
    print(f"Min dimensions: {np.min(widths)} x {np.min(heights)} pixels")
    print(f"Max dimensions: {np.max(widths)} x {np.max(heights)} pixels")
    print(f"Median aspect ratio: {np.median(aspect_ratios):.2f}")
    print(f"Average file size: {np.mean(file_sizes):.1f} KB")
else:
    print("No image dimension data available")

# Calculate imbalance metrics
max_count = df_plants['Count'].max()
min_count = df_plants['Count'].min()
imbalance_ratio = max_count / min_count

print("\033[1m\nIMBALANCE ANALYSIS:\033[0m")
print("")

print(f"Imbalance Ratio: {imbalance_ratio:.2f}:1")
print(f"Standard Deviation: {df_plants['Count'].std():.1f}")

if imbalance_ratio > 5:
    print("SIGNIFICANT CLASS IMBALANCE DETECTED")
    print("This may bias the model toward majority classes")
    print("Recommended: Use class weights or oversampling")
elif imbalance_ratio > 3:
    print("MODERATE CLASS IMBALANCE PRESENT")
else:
    print("Classes are relatively balanced")

**<h2>Class Balance Visuallization</h2>**

In [ ]:
plt.figure(figsize=(16, 10))

# Plot 1: Bar chart with plant names
plt.subplot(2, 1, 1)
bars = plt.bar(range(len(df_plants)), df_plants['Count'], 
               color=['#2E8B57' if x > 200 else '#FFA07A' for x in df_plants['Count']])
plt.title('Distribution of Medicinal Plant Images', fontsize=16, fontweight='bold')
plt.ylabel('Number of Images', fontsize=12)
plt.xticks(range(len(df_plants)), [f"{name}\n({type})" for name, type in zip(df_plants['Plant_Name'], df_plants['Type'])], 
           rotation=45, ha='right', fontsize=10)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, count in zip(bars, df_plants['Count']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
             str(count), ha='center', va='bottom', fontsize=9)

# Plot 2: Pie chart showing imbalance
plt.subplot(2, 1, 2)
plt.pie(df_plants['Count'], labels=df_plants['Plant_Name'], autopct='%1.1f%%', 
        startangle=90, textprops={'fontsize': 8})
plt.title('Percentage Distribution Across Plant Species', fontweight='bold')

plt.tight_layout()
plt.show()

# Detailed statistics
print("\033[1m\nDETAILED CLASS STATISTICS:\n\033[0m")
print(df_plants.describe())

# Identify potential biases
print("\033[1m\nPOTENTIAL BIASES IDENTIFIED:\n\033[0m")
print(f"1. Punarnava has {max_count} images (3.7x more than Shotomuli)")
print(f"2. 6 plants have <150 images, while 4 plants have >250 images")
print(f"3. The model may perform better on well-represented plants like Punarnava, Kalamegh, Syspan")
print(f"4. Rare plants like Shotomuli, Bamonhati may be harder to classify accurately")

**<h2> Sample Images Visualization & Intra-class Variation</h2>**

In [ ]:
def show_samples(class_id, n=4):
    """Display sample images from a specific plant class"""
    path = os.path.join(dataset_path, class_id) 
    all_images = os.listdir(path)
    
    if len(all_images) < n:
        images = all_images
    else:
        images = random.sample(all_images, n)
    
    plant_name = plant_mapping[class_id]  
    
    plt.figure(figsize=(12, 3))
    for i, img in enumerate(images):
        img_path = os.path.join(path, img)
        img_data = Image.open(img_path)
        plt.subplot(1, n, i+1)
        plt.imshow(img_data)
        plt.title(f"{plant_name}\n{class_id}", fontsize=10)
        plt.axis('off')
    
    plt.suptitle(f"{plant_name} ({len(all_images)} images)", fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print observation about this plant's variation
    diversity_level = "good" if len(all_images) > 150 else "moderate" if len(all_images) > 100 else "limited"
    print(f"{plant_name}: Shows {diversity_level} diversity in leaf morphology")

import random

for cid in classes[:16]:  
    show_samples(cid)

**<h2>Image Properties Investigation</h2>**

In [ ]:
# Analyze properties by plant type
plant_properties = {}
all_widths, all_heights, all_aspects, all_sizes = [], [], [], []

for type_name, plant_name in plant_mapping.items():
    if type_name not in class_counts:  # Skip if type doesn't exist
        continue
        
    class_path = os.path.join(dataset_path, type_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    widths, heights, aspects, file_sizes = [], [], [], []
    
    # Sample up to 25 images per plant for analysis
    sample_size = min(25, len(images))
    for img_file in images[:sample_size]:
        img_path = os.path.join(class_path, img_file)
        try:
            with Image.open(img_path) as img:
                width, height = img.size
                widths.append(width)
                heights.append(height)
                aspect = width / height
                aspects.append(aspect)
                
                # Add to global lists
                all_widths.append(width)
                all_heights.append(height)
                all_aspects.append(aspect)
                
            file_size = os.path.getsize(img_path) / 1024  # KB
            file_sizes.append(file_size)
            all_sizes.append(file_size)
            
        except Exception as e:
            continue
    
    if widths:  # If we have valid images
        plant_properties[plant_name] = {
            'avg_width': np.mean(widths),
            'avg_height': np.mean(heights),
            'std_width': np.std(widths),
            'std_height': np.std(heights),
            'min_width': np.min(widths),
            'max_width': np.max(widths),
            'min_height': np.min(heights),
            'max_height': np.max(heights),
            'avg_aspect': np.mean(aspects),
            'std_aspect': np.std(aspects),
            'avg_size_kb': np.mean(file_sizes),
            'total_pixels': np.mean([w*h for w,h in zip(widths, heights)]),
            'resolution_consistency': (np.std(widths) / np.mean(widths) + np.std(heights) / np.mean(heights)) * 100
        }

# Convert to DataFrame for analysis
df_properties = pd.DataFrame(plant_properties).T

print("\033[1m\nDETAILED IMAGE PROPERTIES BY PLANT SPECIES:\n\033[0m")
print(df_properties[['avg_width', 'avg_height', 'avg_aspect', 'avg_size_kb', 'resolution_consistency']].round(2),"\n")

# Create enhanced visualization
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Resolution scatter plot with error bars
for i, plant in enumerate(df_properties.index):
    axes[0,0].errorbar(df_properties.loc[plant, 'avg_width'], 
                      df_properties.loc[plant, 'avg_height'],
                      xerr=df_properties.loc[plant, 'std_width'],
                      yerr=df_properties.loc[plant, 'std_height'],
                      fmt='o', alpha=0.7, label=plant, markersize=8,
                      capsize=3, capthick=1)

axes[0,0].set_title('A) Average Image Dimensions with Variability', fontweight='bold')
axes[0,0].set_xlabel('Width (pixels)')
axes[0,0].set_ylabel('Height (pixels)')
axes[0,0].grid(alpha=0.3)
axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 2. Aspect ratio distribution with variability
y_pos = np.arange(len(df_properties))
axes[0,1].barh(y_pos, df_properties['avg_aspect'], xerr=df_properties['std_aspect'],
               color='lightblue', alpha=0.7, capsize=3)
axes[0,1].set_yticks(y_pos)
axes[0,1].set_yticklabels(df_properties.index)
axes[0,1].axvline(x=1.0, color='red', linestyle='--', alpha=0.7, label='Square (1:1)')
axes[0,1].set_title('B) Aspect Ratio Distribution', fontweight='bold')
axes[0,1].set_xlabel('Aspect Ratio (Width/Height)')
axes[0,1].legend()

# 3. File size distribution
axes[0,2].barh(df_properties.index, df_properties['avg_size_kb'], 
               color='lightgreen', alpha=0.7)
axes[0,2].set_title('C) Average File Size by Species', fontweight='bold')
axes[0,2].set_xlabel('File Size (KB)')

# 4. Resolution consistency (Coefficient of Variation - lower is better)
consistency_sorted = df_properties.sort_values('resolution_consistency')
axes[1,0].barh(consistency_sorted.index, consistency_sorted['resolution_consistency'],
               color='lightcoral', alpha=0.7)
axes[1,0].axvline(x=10, color='red', linestyle='--', label='Good consistency threshold')
axes[1,0].set_title('D) Resolution Consistency\n(Coefficient of Variation % - Lower = Better)', fontweight='bold')
axes[1,0].set_xlabel('CV (%) = (Std/Mean) × 100')
axes[1,0].legend()

# 5. Total pixels (resolution) comparison
axes[1,1].barh(df_properties.index, df_properties['total_pixels'] / 1000, 
               color='plum', alpha=0.7)
axes[1,1].set_title('E) Average Total Pixels (Thousands)', fontweight='bold')
axes[1,1].set_xlabel('Pixels (×1000)')

# 6. Resolution range visualization
for i, plant in enumerate(df_properties.index):
    min_w, max_w = df_properties.loc[plant, 'min_width'], df_properties.loc[plant, 'max_width']
    avg_w = df_properties.loc[plant, 'avg_width']
    axes[1,2].plot([min_w, max_w], [i, i], 'o-', color='blue', alpha=0.7, linewidth=2)
    axes[1,2].plot(avg_w, i, 'o', color='red', markersize=6, label='Average' if i==0 else "")

axes[1,2].set_yticks(range(len(df_properties)))
axes[1,2].set_yticklabels(df_properties.index)
axes[1,2].set_title('F) Width Range by Species\n(Blue=Range, Red=Average)', fontweight='bold')
axes[1,2].set_xlabel('Width (pixels)')
axes[1,2].legend()

plt.tight_layout()
plt.show()

# Enhanced overall statistics
print("\033[1m\nOVERALL RESOLUTION STATISTICS:\n\033[0m")
print(f"Dataset Average: {np.mean(all_widths):.0f} × {np.mean(all_heights):.0f} pixels")
print(f"Resolution Range: {np.min(all_widths)}×{np.min(all_heights)} to {np.max(all_widths)}×{np.max(all_heights)}")
print(f"Aspect Ratio: {np.mean(all_aspects):.2f} ± {np.std(all_aspects):.2f}")
print(f"File Sizes: {np.min(all_sizes):.1f} - {np.max(all_sizes):.1f} KB (avg: {np.mean(all_sizes):.1f} KB)")

# Identify plants with resolution issues
high_variability = df_properties[df_properties['resolution_consistency'] > 15]
if len(high_variability) > 0:
    print("\033[1m\nPLANTS WITH HIGH RESOLUTION VARIABILITY (>15% CV):\n\033[0m")
    for plant in high_variability.index:
        cv = high_variability.loc[plant, 'resolution_consistency']
        print(f"   {plant}: {cv:.1f}% variability")

print("\033[1m\nENHANCED PREPROCESSING STRATEGY RECOMMENDATIONS:\n\033[0m")

# Determine optimal resize based on analysis
avg_resolution = (np.mean(all_widths), np.mean(all_heights))
if avg_resolution[0] > 500 or avg_resolution[1] > 500:
    recommended_size = "256×256 or 224×224"
elif avg_resolution[0] > 300 or avg_resolution[1] > 300:
    recommended_size = "224×224" 
else:
    recommended_size = "128×128"

print(f"1. RESIZING: Standardize to {recommended_size} for optimal CNN performance")
print(f"   Current average: {np.mean(all_widths):.0f}×{np.mean(all_heights):.0f}")
print(f"   Resizing will reduce computational requirements while maintaining features")

print(f"2. NORMALIZATION: Use ImageNet statistics for transfer learning")
print(f"   Mean: [0.485, 0.456, 0.406], Std: [0.229, 0.224, 0.225]")

print(f"3. DATA AUGMENTATION:")
print(f"   Account for natural resolution variations")
print(f"   Use center cropping for consistent input sizes")
print(f"   Maintain aspect ratio during initial processing")

print(f"4. QUALITY CONTROL:")
print(f"   Monitor plants with high resolution variability: {', '.join(high_variability.index) if len(high_variability) > 0 else 'None'}")
print(f"   Ensure consistent preprocessing across all species")

print(f"5. MEMORY OPTIMIZATION:")
print(f"   Average file size: {np.mean(all_sizes):.1f} KB -> Efficient for batch processing")
print(f"   Recommended batch size: 32-64 for 224×224 images")

# Technical insights for model selection
print("\033[1m\nTECHNICAL INSIGHTS FOR MODEL DEVELOPMENT:\n\033[0m")
print(f"Input shape compatibility: All major CNN architectures support {recommended_size} inputs")
print(f"Memory requirements: ~{len(plant_mapping) * 50}MB for model + features")
print(f"Training time: Moderate (2-4 hours on GPU for full fine-tuning)")
print(f"Potential issues: Resolution variability may affect some species more than others")


**<h2>COLOR DISTRIBUTION ANALYSIS</h2>**

In [ ]:
print("\033[1m\nCOLOR DISTRIBUTION ANALYSIS:\n\033[0m")
R, G, B = [], [], []
color_by_plant = {}

for class_name in classes:
    class_path = os.path.join(dataset_path, class_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # Use plant name from mapping instead of undefined class_names
    plant_name = plant_mapping.get(class_name, class_name)
    
    plant_R, plant_G, plant_B = [], [], []
    
    # Sample up to 8 images per plant
    sample_images = images[:8]
    
    for imgname in sample_images:
        try:
            img_path = os.path.join(class_path, imgname)
            img = Image.open(img_path).convert('RGB')
            arr = np.array(img)/255.0
            plant_R.append(arr[:,:,0].mean())
            plant_G.append(arr[:,:,1].mean())
            plant_B.append(arr[:,:,2].mean())
            
            R.append(arr[:,:,0].mean())
            G.append(arr[:,:,1].mean())
            B.append(arr[:,:,2].mean())
        except Exception as e:
            continue
    
    if plant_R:  # If we have color data for this plant
        color_by_plant[plant_name] = {
            'R': np.mean(plant_R),
            'G': np.mean(plant_G), 
            'B': np.mean(plant_B)
        }

# Create enhanced color visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot for overall color distribution
box_data = [R, G, B]
axes[0].boxplot(box_data, labels=['Red', 'Green', 'Blue'])
axes[0].set_title("Overall Color Channel Distribution", fontweight='bold')
axes[0].set_ylabel("Mean Pixel Value (0-1)")
axes[0].grid(alpha=0.3)

# Plant-specific color analysis
plant_names = list(color_by_plant.keys())
plant_r = [color_by_plant[p]['R'] for p in plant_names]
plant_g = [color_by_plant[p]['G'] for p in plant_names] 
plant_b = [color_by_plant[p]['B'] for p in plant_names]

x_pos = np.arange(len(plant_names))
width = 0.25

axes[1].bar(x_pos - width, plant_r, width, label='Red', color='red', alpha=0.7)
axes[1].bar(x_pos, plant_g, width, label='Green', color='green', alpha=0.7)
axes[1].bar(x_pos + width, plant_b, width, label='Blue', color='blue', alpha=0.7)

axes[1].set_title("Average Color Values by Plant Species", fontweight='bold')
axes[1].set_xlabel("Plant Species")
axes[1].set_ylabel("Mean Color Value")
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(plant_names, rotation=45, ha='right')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\033[1mColor Analysis Insights:\033[0m")
print(f"Green channel dominates (mean: {np.mean(G):.3f}) - typical for plant leaves")
print(f"Color variance across plants: R={np.std(R):.3f}, G={np.std(G):.3f}, B={np.std(B):.3f}")
print(f"Some plants show distinct color profiles useful for identification")

**<h2>ADVANCED COLOR SPACE ANALYSIS (HSV & LAB)</h2>**

In [ ]:
print("\033[1m\nADVANCED COLOR SPACE ANALYSIS (HSV & LAB):\n\033[0m")
print("HSV & LAB color spaces can reveal plant health and species-specific characteristics")

H_vals, S_vals, V_vals = [], [], []
L_vals, A_vals, B_vals = [], [], []
color_by_plant_advanced = {}

for type_name, plant_name in plant_mapping.items():
    if type_name not in class_counts:  # Skip if type doesn't exist
        continue
        
    class_path = os.path.join(dataset_path, type_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    plant_H, plant_S, plant_V = [], [], []
    plant_L, plant_A, plant_B = [], [], []
    
    # Sample 5 images per plant
    sample_images = images[:5]
    
    for img_name in sample_images:
        try:
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Convert to HSV and LAB
            hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
            lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
            
            # Collect channel means
            plant_H.append(hsv[:,:,0].mean())
            plant_S.append(hsv[:,:,1].mean()) 
            plant_V.append(hsv[:,:,2].mean())
            plant_L.append(lab[:,:,0].mean())
            plant_A.append(lab[:,:,1].mean())
            plant_B.append(lab[:,:,2].mean())
            
            # Add to overall lists
            H_vals.append(hsv[:,:,0].mean())
            S_vals.append(hsv[:,:,1].mean())
            V_vals.append(hsv[:,:,2].mean())
            L_vals.append(lab[:,:,0].mean())
            A_vals.append(lab[:,:,1].mean())
            B_vals.append(lab[:,:,2].mean())
            
        except Exception as e:
            continue
    
    if plant_H:  # If we have data for this plant
        color_by_plant_advanced[plant_name] = {
            'H': np.mean(plant_H), 'S': np.mean(plant_S), 'V': np.mean(plant_V),
            'L': np.mean(plant_L), 'A': np.mean(plant_A), 'B': np.mean(plant_B)
        }

# Create comprehensive color space visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# HSV Distribution
hsv_data = [H_vals, S_vals, V_vals]
axes[0,0].boxplot(hsv_data, labels=['Hue', 'Saturation', 'Value'])
axes[0,0].set_title("HSV Color Space Distribution", fontweight='bold')
axes[0,0].set_ylabel("Mean Channel Value")
axes[0,0].grid(alpha=0.3)

# LAB Distribution  
lab_data = [L_vals, A_vals, B_vals]
axes[0,1].boxplot(lab_data, labels=['L* (Lightness)', 'a* (Green-Red)', 'b* (Blue-Yellow)'])
axes[0,1].set_title("LAB Color Space Distribution", fontweight='bold')
axes[0,1].set_ylabel("Mean Channel Value")
axes[0,1].grid(alpha=0.3)

# Plant-specific Saturation (indicator of leaf health/vibrancy)
plant_names = list(color_by_plant_advanced.keys())
saturation_vals = [color_by_plant_advanced[p]['S'] for p in plant_names]

axes[1,0].barh(plant_names, saturation_vals, color='green', alpha=0.7)
axes[1,0].set_title("Average Saturation by Plant Species", fontweight='bold')
axes[1,0].set_xlabel("Saturation Value")
axes[1,0].grid(alpha=0.3)

# Plant-specific A channel (green-red balance)
a_channel_vals = [color_by_plant_advanced[p]['A'] for p in plant_names]

axes[1,1].barh(plant_names, a_channel_vals, color='red', alpha=0.7)
axes[1,1].axvline(x=128, color='black', linestyle='--', label='Neutral (128)')
axes[1,1].set_title("A* Channel (Green-Red Balance) by Plant", fontweight='bold')
axes[1,1].set_xlabel("A* Value (Lower = Greener)")
axes[1,1].legend()
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\033[1m\nADVANCED COLOR INSIGHTS:\033[0m")
print(f"Saturation Range: {np.min(S_vals):.1f}-{np.max(S_vals):.1f} (indicates leaf vibrancy)")
print(f"A* Channel Mean: {np.mean(A_vals):.1f} (lower values = more green dominance)")
print(f"Color spaces reveal subtle species differences not visible in RGB")

**<h2>Augmentation Analysis for Medicinal Plants</h2>**

In [ ]:
print("\033[1m\nAUGMENTATION PROBE FOR MEDICINAL PLANT IDENTIFICATION:\n\033[0m")
# Test on multiple plant types to see generalization
test_plants = ['Kalochitra', 'Punarnava', 'Shotomuli']  # Diverse examples

fig, axes = plt.subplots(len(test_plants), 7, figsize=(20, 3*len(test_plants)))

if len(test_plants) == 1:
    axes = [axes]

for row, plant_name in enumerate(test_plants):
    # Find the type for this plant
    type_name = [k for k, v in plant_mapping.items() if v == plant_name][0]
    class_path = os.path.join(dataset_path, type_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if images:
        sample_path = os.path.join(class_path, images[0])
        original_img = Image.open(sample_path)
        
        # Define augmentations
        augmentations = {
            'Original': original_img,
            'Horizontal Flip': original_img.transpose(Image.FLIP_LEFT_RIGHT),
            'Vertical Flip': original_img.transpose(Image.FLIP_TOP_BOTTOM),
            'Rotation (15°)': original_img.rotate(15, expand=True),
            'Rotation (45°)': original_img.rotate(45, expand=True),
            'Brightness +30%': Image.eval(original_img, lambda x: min(255, x * 1.3)),
            'Color Jitter': Image.eval(original_img, lambda x: min(255, (x - 128) * 1.2 + 128)),
        }
        
        for col, (aug_name, aug_img) in enumerate(augmentations.items()):
            axes[row][col].imshow(aug_img)
            if row == 0:  # Title only on first row
                axes[row][col].set_title(aug_name, fontweight='bold', fontsize=10)
            axes[row][col].axis('off')
            if col == 0:  # Plant name on first column
                axes[row][col].text(-0.3, 0.5, plant_name, transform=axes[row][col].transAxes,
                                  fontsize=12, fontweight='bold', va='center', ha='right')

plt.suptitle('Augmentation Effects on Different Medicinal Plants\n\n', fontsize=16, fontweight='bold', y=0.95)
plt.tight_layout()
plt.show()

# Safety Analysis
print("""
\033[1mSAFE AUGMENTATIONS:\033[0m

  Horizontal Flip 
  - Preserves leaf symmetry and morphology
  - Maintains medicinal identification features
  
  Small Rotations (±15-20°)
  - Represents natural leaf orientation variations
  - Doesn't distort key morphological features
  
  Brightness/Contrast Adjustments (±20-30%)
  - Simulates different lighting conditions
  - Maintains color integrity for identification

  Mild Color Preservation
  - Small saturation adjustments
  - Maintains species-specific color patterns

\033[1mCAUTION REQUIRED AUGMENTATIONS:\033[0m

  Large Rotations (>30°)
  - May change leaf orientation semantics
  - Could affect vein pattern recognition
  
  Vertical Flip
  - Unnatural for leaf images
  - May confuse some CNN architectures

  Extreme Cropping
  - Might remove important leaf parts
  - Could eliminate diagnostic features

\033[1mPOTENTIALLY HARMFUL AUGMENTATIONS:\033[0m

  Extreme Color Jitter
  - Could alter important color-based identification
  - May change leaf appearance significantly
  
  Shearing/Warping
  - Distorts natural leaf morphology
  - Affects shape-based classification
  
  Extreme Brightness (>50% change)
  - Washes out important texture details
  - Reduces feature discriminability
""")

print("\033[1m\nFINAL AUGMENTATION STRATEGY:\033[0m")
print("")
print("• Use: Horizontal flips, small rotations, mild brightness/contrast")
print("• Avoid: Vertical flips, extreme color changes, heavy cropping")
print("• Test: Monitor validation performance with different augmentations")
print("• Balance: Combine safe augmentations for robust model training")

**<h2>ENHANCED AUGMENTATION & ROBUSTNESS ANALYSIS</h2>**

In [ ]:
def add_gaussian_noise(image, mean=0, sigma=25):
    """Add Gaussian noise to test model robustness"""
    noise = np.random.normal(mean, sigma, image.shape).astype(np.int16)
    noisy_img = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return noisy_img

def analyze_background_dominance(img_path):
    """Analyze how much of the image is background vs plant"""
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Calculate background percentage (assuming light backgrounds)
    background_pixels = np.sum(gray > 200)  # threshold for light background
    percent_bg = (background_pixels / gray.size) * 100
    
    return percent_bg

# Test on representative samples
test_samples = [
    ('type_1', 'Kalochitra'),
    ('type_6', 'Kalamegh'), 
    ('type_11', 'Punarnava'),
    ('type_15', 'Shotomuli')  # Include a minority class
]

# Background analysis
print("\033[1mBACKGROUND ANALYSIS:\033[0m")
print("")
bg_percentages = []

for type_name, plant_name in test_samples:
    class_path = os.path.join(dataset_path, type_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if images:
        img_path = os.path.join(class_path, images[0])
        bg_percent = analyze_background_dominance(img_path)
        bg_percentages.append(bg_percent)
        status = "LOW" if bg_percent < 30 else "HIGH"
        print(f"  {plant_name}: {bg_percent:.1f}% background ({status})")

plt.figure(figsize=(10, 5))
plt.hist(bg_percentages, bins=15, color='lightblue', edgecolor='black', alpha=0.7)
plt.axvline(x=30, color='red', linestyle='--', label='Ideal Threshold (30%)')
plt.title("Background Dominance Distribution", fontweight='bold')
plt.xlabel("Background Percentage (%)")
plt.ylabel("Number of Samples")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Comprehensive augmentation robustness test
print("\033[1m\nROBUSTNESS TESTING WITH ADVANCED AUGMENTATIONS:\033[0m")
print("")

for type_name, plant_name in test_samples[:2]:  # Test on 2 plants
    class_path = os.path.join(dataset_path, type_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if images:
        img_path = os.path.join(class_path, images[0])
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Create various augmentations
        from torchvision import transforms as T
        
        transform_flip = T.RandomHorizontalFlip(p=1)
        transform_rot = T.RandomRotation(20)
        transform_color = T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2)
        transform_pil = T.ToPILImage()
        
        # Convert to PIL for torchvision transforms
        img_pil = transform_pil(img_rgb)
        
        # Apply augmentations
        flipped = transform_flip(img_pil)
        rotated = transform_rot(img_pil)
        color_jittered = transform_color(img_pil)
        
        # Advanced augmentations
        noisy_img = add_gaussian_noise(img_rgb)
        
        # Zoom augmentation (center crop + resize)
        h, w = img_rgb.shape[:2]
        crop_ratio = 0.15
        crop = img_rgb[int(h*crop_ratio):int(h*(1-crop_ratio)), 
                       int(w*crop_ratio):int(w*(1-crop_ratio))]
        zoomed = cv2.resize(crop, (w, h))
        
        # Display all augmentations
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        
        augmentations = [
            ("Original", img_rgb),
            ("Horizontal Flip", np.array(flipped)),
            ("Rotation 20°", np.array(rotated)),
            ("Color Jitter", np.array(color_jittered)),
            ("Gaussian Noise", noisy_img),
            ("Zoom (15% crop)", zoomed),
            ("Brightness +40%", np.array(T.ColorJitter(brightness=0.4)(img_pil))),
            ("Contrast +40%", np.array(T.ColorJitter(contrast=0.4)(img_pil)))
        ]
        
        for i, (title, aug_img) in enumerate(augmentations):
            row, col = i // 4, i % 4
            axes[row, col].imshow(aug_img)
            axes[row, col].set_title(title, fontsize=10, fontweight='bold')
            axes[row, col].axis('off')
        
        plt.suptitle(f"Advanced Augmentation Robustness: {plant_name}", fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

print("\033[1m\nENHANCED AUGMENTATION STRATEGY:\033[0m")
print("")
print("SAFE & RECOMMENDED:")
print("•Horizontal Flips")
print("• Small Rotations (15-25°)")
print("• Mild Color Jitter (brightness/contrast ±30%, saturation ±20%)")
print("• Gaussian Noise (σ=15-25) - improves robustness")
print("• Center Zoom (10-15% crop) - focuses on leaf details")
print("")
print("USE JUDICIOUSLY:")
print("• Larger Rotations (>30°) - monitor performance")
print("• Heavy Color Changes - may affect species identification")
print("• Extreme Cropping (>20%) - might remove key features")
print("")
print("INSIGHTS FROM ADVANCED ANALYSIS:")
print("• Backgrounds are generally clean (<30% background)")
print("• HSV/LAB spaces reveal subtle species differences")
print("• Noise robustness can be built with Gaussian augmentation")
print("• Most plants maintain identifiable features under reasonable transformations")